# Analisis Sentimen pada Ulasan 10 Film Indonesia yang Dikumpulkan dari Letterboxd

**Latar Belakang Masalah**

Sebagai seorang yang tertarik dengan persimpangan antara teknologi dan industri perfilman, saya mengamati bahwa jumlah ulasan film online terus bertambah secara eksponensial. Namun, seringkali sulit untuk mendapatkan gambaran ringkas dan objektif tentang sentimen penonton terhadap suatu film hanya dengan membaca ulasan satu per satu. Hal ini memotivasi saya untuk melakukan proyek analisis sentimen ini.

**Tujuan Analisis**

Tujuan utama saya adalah mengembangkan sebuah sistem yang mampu secara otomatis mengklasifikasikan sentimen dalam ulasan film, mengubah data tekstual yang luas menjadi informasi yang terstruktur dan mudah dipahami. Dengan demikian, proyek ini diharapkan dapat memberikan insight yang berharga bagi para pembuat film, kritikus, dan penonton dalam memahami reaksi publik terhadap film-film Indonesia.



# Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

Tahap ini melibatkan pengimporan library-library Python yang diperlukan untuk melakukan web scraping. requests digunakan untuk mengirimkan permintaan HTTP ke website, beautifulsoup4 untuk mem-parse konten HTML, pandas untuk membuat dan memanipulasi DataFrame, time untuk memberikan jeda antar permintaan, dan random untuk membuat jeda acak.

# Scrapping Ulasan


In [2]:
daftar_film = [
    {"title": "Laskar Pelangi", "url": "https://letterboxd.com/film/laskar-pelangi/reviews/by/activity/page/{}/"},
    {"title": "Sang Pemimpi", "url": "https://letterboxd.com/film/sang-pemimpi/reviews/by/activity/page/{}/"},
    {"title": "The Raid: Redemption", "url": "https://letterboxd.com/film/the-raid-redemption/reviews/by/activity/page/{}/"},
    {"title": "Habibie & Ainun", "url": "https://letterboxd.com/film/habibie-ainun/reviews/by/activity/page/{}/"},
    {"title": "Pengabdi Setan", "url": "https://letterboxd.com/film/pengabdi-setan/reviews/by/activity/page/{}/"},
    {"title": "Dilan 1990", "url": "https://letterboxd.com/film/dilan-1990/reviews/by/activity/page/{}/"},
    {"title": "5cm", "url": "https://letterboxd.com/film/5-cm/reviews/by/activity/page/{}/"},
    {"title": "KKN di Desa Penari", "url": "https://letterboxd.com/film/kkn-curse-of-the-dancing-village/reviews/by/activity/page/{}/"},
    {"title": "Gundala", "url": "https://letterboxd.com/film/gundala/reviews/by/activity/page/{}/"},
    {"title": "Mencuri Raden Saleh", "url": "https://letterboxd.com/film/stealing-raden-saleh/reviews/by/activity/page/{}/"},
]

Pada tahap ini, kita mendefinisikan sebuah daftar yang berisi informasi tentang film-film yang akan di-scrape. Setiap film direpresentasikan sebagai dictionary yang mencakup judul film (title) dan URL dasar untuk ulasannya di Letterboxd (url). URL ini diformat untuk memungkinkan pagination (pengambilan halaman berikutnya).

In [3]:
def ambil_halaman(url, headers=None, timeout=10, max_coba=3, jeda=2):
    """Mengambil konten halaman web dengan penanganan kesalahan."""

    if headers is None:
        headers = {"User-Agent": "Mozilla/5.0"}

    for coba in range(max_coba):
        try:
            respons = requests.get(url, headers=headers, timeout=timeout)
            respons.raise_for_status()
            return BeautifulSoup(respons.content, "html.parser")
        except requests.exceptions.RequestException as e:
            print(f"Gagal mengambil {url}, mencoba lagi ({coba + 1}/{max_coba})... Alasan: {e}")
            if coba < max_coba - 1:
                time.sleep(jeda * (coba + 1))
            else:
                print(f"Gagal mengambil {url} setelah {max_coba} kali percobaan.")
                return None
    return None

Fungsi ambil_halaman() bertanggung jawab untuk mengambil konten HTML dari URL yang diberikan. Fungsi ini dilengkapi dengan error handling untuk menangani kegagalan permintaan HTTP, mekanisme retry dengan exponential backoff, dan pengaturan User-Agent untuk meniru browser. Jika pengambilan halaman berhasil, fungsi ini mengembalikan objek BeautifulSoup yang berisi konten HTML yang di-parse.

In [4]:
def ekstrak_ulasan_dari_halaman(soup, film_title):
    """Mengekstrak data ulasan dari objek BeautifulSoup."""

    ulasan = []

    blok_ulasan = soup.find_all("article", class_="production-record")

    if not blok_ulasan:

        print(f"Tidak ditemukan blok ulasan di halaman ini untuk film '{film_title}'.")
        return ulasan

    for blok in blok_ulasan:
        try:


            nama_pengguna = "Anonim"
            nama_pengguna_elemen = blok.find("div", class_="avatar")
            if nama_pengguna_elemen:
                link_pengguna = nama_pengguna_elemen.find("a")
                if link_pengguna and "data-original-title" in link_pengguna.attrs:
                    nama_pengguna = link_pengguna["data-original-title"].strip()
                elif link_pengguna:
                    nama_pengguna = link_pengguna.text.strip()

            teks_ulasan_elemen = blok.find("div", class_="body-text")
            teks_ulasan = teks_ulasan_elemen.text.strip() if teks_ulasan_elemen else "Tidak ada ulasan"

            rating = "Tidak ada rating"
            rating_elemen = blok.find("span", class_="rating")
            if rating_elemen:

                if "title" in rating_elemen.attrs:
                    rating = rating_elemen["title"].strip()
                else:

                    for cls in rating_elemen.get("class", []):
                        if cls.startswith("rated-"):
                            try:

                                numeric_rating_str = cls.split("-")[1]
                                numeric_rating = float(numeric_rating_str) / 2
                                rating = f"{numeric_rating} stars"
                                break
                            except ValueError:
                                pass

                    if rating == "Tidak ada rating":
                        rating = rating_elemen.text.strip()

            ulasan.append(
                {
                    "film": film_title,
                    "pengguna": nama_pengguna,
                    "ulasan": teks_ulasan,
                    "rating": rating,
                }
            )
        except Exception as e:
            print(f"Gagal mengekstrak beberapa elemen dari blok ulasan untuk film '{film_title}'. Kesalahan: {e}")


            continue

    return ulasan

 Fungsi ekstrak_ulasan_dari_halaman() mem-parse konten HTML yang diberikan (dalam bentuk objek BeautifulSoup) dan mengekstrak informasi ulasan seperti username, teks ulasan, dan rating. Fungsi ini menggunakan metode find_all() dan find() dari BeautifulSoup untuk menemukan elemen HTML yang relevan. Error handling diterapkan menggunakan blok try-except untuk menangani kasus di mana elemen HTML tertentu tidak ditemukan.

In [5]:
def kumpulkan_ulasan_film(
    daftar_film, halaman_maksimal=200, jeda_antar_halaman_min=1, jeda_antar_halaman_maks=3
):
    """Mengumpulkan ulasan dari beberapa film."""

    semua_ulasan = []

    for film in daftar_film:
        judul_film = film["title"]
        url_film = film["url"]

        print(f"Memulai scraping ulasan untuk '{judul_film}'...")

        for nomor_halaman in range(1, halaman_maksimal + 1):
            url_halaman = url_film.format(nomor_halaman)
            print(f"Mengambil halaman: {url_halaman}")

            soup = ambil_halaman(url_halaman)
            if soup:
                ulasan_halaman = ekstrak_ulasan_dari_halaman(soup, judul_film)

                if ulasan_halaman:
                    semua_ulasan.extend(ulasan_halaman)
                else:

                    print(f"Tidak ada ulasan ditemukan di halaman {nomor_halaman} untuk {judul_film}. Berhenti scraping untuk film ini.")
                    break
            else:
                print(f"Gagal mengambil halaman {nomor_halaman} untuk {judul_film}, melanjutkan...")
                continue


            jeda_acak = random.uniform(jeda_antar_halaman_min, jeda_antar_halaman_maks)
            time.sleep(jeda_acak)

        print(f"Selesai scraping ulasan untuk '{judul_film}'. Total ulasan: {len(semua_ulasan)}.\n")

    return pd.DataFrame(semua_ulasan)

 Fungsi kumpulkan_ulasan_film() adalah fungsi utama yang mengkoordinasikan proses scraping. Fungsi ini melakukan iterasi melalui daftar film, membuat URL halaman ulasan untuk setiap film, memanggil ambil_halaman() untuk mendapatkan konten HTML, dan memanggil ekstrak_ulasan_dari_halaman() untuk mengekstrak ulasan. Fungsi ini juga menerapkan random delay menggunakan time.sleep() untuk menghindari pemblokiran dan mematuhi etika scraping. Semua ulasan yang diekstrak disimpan dalam sebuah list dan akhirnya dikembalikan sebagai DataFrame pandas.

In [6]:
if __name__ == "__main__":
    df_ulasan = kumpulkan_ulasan_film(daftar_film)

    if not df_ulasan.empty:
        nama_file_csv = "ulasan_film_indonesia.csv"
        df_ulasan.to_csv(nama_file_csv, index=False, encoding="utf-8")
        print(f"Ulasan berhasil disimpan ke dalam '{nama_file_csv}'.")
        print(f"Jumlah total ulasan yang dikumpulkan: {len(df_ulasan)}")
    else:
        print("Tidak ada ulasan yang berhasil dikumpulkan.")

Memulai scraping ulasan untuk 'Laskar Pelangi'...
Mengambil halaman: https://letterboxd.com/film/laskar-pelangi/reviews/by/activity/page/1/
Mengambil halaman: https://letterboxd.com/film/laskar-pelangi/reviews/by/activity/page/2/
Mengambil halaman: https://letterboxd.com/film/laskar-pelangi/reviews/by/activity/page/3/
Mengambil halaman: https://letterboxd.com/film/laskar-pelangi/reviews/by/activity/page/4/
Mengambil halaman: https://letterboxd.com/film/laskar-pelangi/reviews/by/activity/page/5/
Mengambil halaman: https://letterboxd.com/film/laskar-pelangi/reviews/by/activity/page/6/
Mengambil halaman: https://letterboxd.com/film/laskar-pelangi/reviews/by/activity/page/7/
Mengambil halaman: https://letterboxd.com/film/laskar-pelangi/reviews/by/activity/page/8/
Mengambil halaman: https://letterboxd.com/film/laskar-pelangi/reviews/by/activity/page/9/
Mengambil halaman: https://letterboxd.com/film/laskar-pelangi/reviews/by/activity/page/10/
Mengambil halaman: https://letterboxd.com/film/la

Blok kode ini dijalankan ketika script Python dieksekusi secara langsung (bukan ketika diimpor sebagai module). Kode ini memanggil kumpulkan_ulasan_film() untuk memulai proses scraping, dan jika ulasan berhasil dikumpulkan (DataFrame tidak kosong), kode ini menyimpan ulasan tersebut ke dalam file CSV dengan nama ulasan_film_indonesia.csv.

In [7]:
df = pd.read_csv("ulasan_film_indonesia.csv")
df

,film,pengguna,ulasan,rating
0,Laskar Pelangi,Anonim,I love this so much,4.5 stars
1,Laskar Pelangi,Anonim,Ma kinda childhood movie 🌻,4.0 stars
2,Laskar Pelangi,Anonim,"This movie is just beautiful, touching, inspir...",4.5 stars
3,Laskar Pelangi,Anonim,read the book & watched this in middle school ...,3.0 stars
4,Laskar Pelangi,Anonim,Film legendaris Indonesia yang sangat jelas di...,4.5 stars
...,...,...,...,...
15443,Mencuri Raden Saleh,Anonim,I truly love how they messed up the plan sever...,4.0 stars
15444,Mencuri Raden Saleh,Anonim,Worth the hype kata gw mah,4.5 stars
15445,Mencuri Raden Saleh,Anonim,ketika lu berfikir untuk nyolong lukisan terma...,5.0 stars
15446,Mencuri Raden Saleh,Anonim,Fun heist movie.Kinda monumental for Indonesia...,3.0 stars
